In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

# Define the modified VGGNet
class VGGModified(nn.Module):
    def __init__(self, num_classes=10, use_dropout=False):
        super(VGGModified, self).__init__()
        self.use_dropout = use_dropout
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5) if use_dropout else nn.Identity(),

            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5) if use_dropout else nn.Identity(),

            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Function to count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

##############3Compare parameter counts
original_vgg16_params = 138000000  #
modified_vgg_params = count_parameters(VGGModified(num_classes=10, use_dropout=False))
print(f"Original VGG-16 Parameters: {original_vgg16_params}")
print(f"Modified VGG Parameters: {modified_vgg_params}")

#######Load CIFAR-10 & CIFAR-100 datasets
def get_data_loaders(batch_size=128):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset_10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset_10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    trainset_100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    testset_100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
    return DataLoader(trainset_10, batch_size=batch_size, shuffle=True), DataLoader(testset_10, batch_size=batch_size, shuffle=False), DataLoader(trainset_100, batch_size=batch_size, shuffle=True), DataLoader(testset_100, batch_size=batch_size, shuffle=False)

##### Training & evaluation function
def train_model(model, trainloader, testloader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(trainloader))

        model.eval()
        correct, total, val_loss = 0, 0, 0.0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_losses.append(val_loss / len(testloader))
        val_accuracies.append(correct / total * 100)
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%")

    return train_losses, val_losses, val_accuracies

#Train and compare results
trainloader_10, testloader_10, trainloader_100, testloader_100 = get_data_loaders()
print("Training VGGNet Without Dropout on CIFAR-10")
train_losses_no_dropout, val_losses_no_dropout, val_acc_no_dropout = train_model(VGGModified(num_classes=10, use_dropout=False), trainloader_10, testloader_10)
print("Training VGGNet With Dropout on CIFAR-10")
train_losses_dropout, val_losses_dropout, val_acc_dropout = train_model(VGGModified(num_classes=10, use_dropout=True), trainloader_10, testloader_10)

print("Training VGGNet Without Dropout on CIFAR-100")
train_losses_no_dropout_100, val_losses_no_dropout_100, val_acc_no_dropout_100 = train_model(VGGModified(num_classes=100, use_dropout=False), trainloader_100, testloader_100)

print("Training VGGNet With Dropout on CIFAR-100")
train_losses_dropout_100, val_losses_dropout_100, val_acc_dropout_100 = train_model(VGGModified(num_classes=100, use_dropout=True), trainloader_100, testloader_100)


######Result in Tableee
results_df = pd.DataFrame({
    "Dataset": ["CIFAR-10", "CIFAR-10", "CIFAR-100", "CIFAR-100"],
    "Dropout": ["No", "Yes", "No", "Yes"],
    "Final Val Accuracy (%)": [val_acc_no_dropout[-1], val_acc_dropout[-1], val_acc_no_dropout_100[-1], val_acc_dropout_100[-1]],
    "Final Val Loss": [val_losses_no_dropout[-1], val_losses_dropout[-1], val_losses_no_dropout_100[-1], val_losses_dropout_100[-1]]
})
print(results_df)



Original VGG-16 Parameters: 138000000
Modified VGG Parameters: 5870666


100%|██████████| 170M/170M [00:02<00:00, 57.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


100%|██████████| 169M/169M [00:01<00:00, 105MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Training VGGNet Without Dropout on CIFAR-10
Epoch 1/10 - Train Loss: 1.7151, Val Loss: 1.3681, Val Acc: 49.89%
Epoch 2/10 - Train Loss: 1.1900, Val Loss: 1.0580, Val Acc: 62.40%
Epoch 3/10 - Train Loss: 0.9527, Val Loss: 0.8564, Val Acc: 69.72%
Epoch 4/10 - Train Loss: 0.8005, Val Loss: 0.7886, Val Acc: 72.57%
Epoch 5/10 - Train Loss: 0.6916, Val Loss: 0.6864, Val Acc: 76.60%
Epoch 6/10 - Train Loss: 0.6191, Val Loss: 0.6139, Val Acc: 78.96%
Epoch 7/10 - Train Loss: 0.5599, Val Loss: 0.6174, Val Acc: 79.07%
Epoch 8/10 - Train Loss: 0.5136, Val Loss: 0.5721, Val Acc: 80.93%
Epoch 9/10 - Train Loss: 0.4822, Val Loss: 0.5900, Val Acc: 80.42%
Epoch 10/10 - Train Loss: 0.4523, Val Loss: 0.5311, Val Acc: 81.93%
Training VGGNet With Dropout on CIFAR-10
Epoch 1/10 - Train Loss: 1.8088, Val Loss: 1.5187, Val Acc: 42.10%
Epoch 2/10 - Train Loss: 1.3316, Val Loss: 1.1319, Val Acc: 59.12%
Epoch 3/10 - Train L